In [1]:
import glob
import MeCab
import ipadic
from gensim.models import Word2Vec
import re
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [6]:
files = glob.glob("sample_data/*.txt")

documents = []

for file_path in files:
    with open(file_path, "r", encoding="utf-8") as f:
        documents.append(f.read())
        
stop_words = []
with open("stop_words.txt", "r", encoding="utf-8") as f:
    for row in f.readlines():
        stop_words.append(row.strip()) 
document_list = []

m = MeCab.Tagger(ipadic.MECAB_ARGS)
pattern = re.compile(r"[ぁ-んァ-ヶ一-龥々]")

for document in documents:
    m1 = m.parse(document)
    
    noun_list =[]
    for row in m1.split("\n"):
        target_word = row.split("\t")[0]
        if target_word == "EOS":
            break
        else:
            word = row.split("\t")[1]
            if word[:2] == "名詞":
                if pattern.findall(target_word) != [] and target_word not in stop_words:
                   noun_list.append(target_word)

    document_list.append(noun_list)

In [ ]:
tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(document_list)]

In [10]:
tagged_data

[TaggedDocument(words=['悪評', '川越', 'シェフ', '最近', '人気', '川越', 'シェフ', '代官山', 'おしゃれ', 'イタリアン', 'レストラン', 'オーナー', 'シェフ', 'タレント', '軽妙', 'トーク', 'マルチ', '才能', '業界', '地位', '不動', 'マスク', '川越', '女子', '固定', 'ファン', '存在', '川越', 'シェフ', '悪評', '噴出', '最近', 'ミニストップ', '川越', 'シェフプロデュースクリスマスケーキ', '高額', '話題', '関連', '記事', 'ミニストップ', '川越', 'シェフプロデュースクリスマスケーキ', 'バカ', '話題', '普通', 'ケーキ', '原価', '程度', '川越', 'シェフ', 'プロデュース', '豪華', 'くり', 'ネット', '反応', '有名', '途端', '金儲け', '声', '散見', '普段', 'マスコミ', '対応', 'さ', 'かなり', '川越', 'シェフ', '状態', '有名', '手の平', '週刊', '記者', '態度', 'マスコミ', 'おかげ', '有名', '恩知らず', '関連', '記事', 'シェフ', 'タレント', 'マスコミ', 'タレント', 'シェフ', 'シェフ', '風情', '調子', 'の', '無理', '株式会社', '年商', '出演', '料', '高額', '番組', 'バツ', '独身', '告白', 'マスク', '有名人', 'モテ', 'カネ', 'オンナ', '放題', 'の', '関連', '記事', '離婚', '告白', '川越', '達也', 'シェフ', 'ネイルサロン', '収録', '口紅', '川越', 'シェフ', '才', '年下', '美人', '秘書', 'シェフ', 'デキ', '説', '否定', '天狗', 'っぷり', '盛ん', '報道', '最近', '収録', '現場', 'ブチギレ', '現場', 'スタッフ', '評判', 'の', '関連', '記事', '川越', 'シェフ', '我慢', '限界', 'ブチギレ', '肝心', '料理', '腕'

In [15]:
model = Doc2Vec(documents=tagged_data, min_count=1, epochs=100)

In [16]:
model.dv.most_similar([model.dv[38]], topn=5)

[('38', 1.0),
 ('40', 0.8934699892997742),
 ('20', 0.5427135825157166),
 ('66', 0.5392549633979797),
 ('71', 0.4835440516471863)]

In [19]:
def tokenize(document):
    m1 = m.parse(document)
    
    noun_list =[]
    for row in m1.split("\n"):
        target_word = row.split("\t")[0]
        if target_word == "EOS":
            break
        else:
            word = row.split("\t")[1]
            if word[:2] == "名詞":
                if pattern.findall(target_word) != [] and target_word not in stop_words:
                   noun_list.append(target_word)
                   
    return noun_list


In [20]:
new_doc = "新しい文章が入ります"
tokenizse_doc = tokenize(new_doc)

vector = model.infer_vector(tokenizse_doc)

In [21]:
vector

array([-0.11268634, -0.10093855,  0.0037616 ,  0.04934754,  0.02909614,
       -0.23325065,  0.09484451,  0.22914809, -0.27479532,  0.15030235,
       -0.1892293 , -0.32922244, -0.08946063,  0.00996154,  0.01694759,
       -0.15256616,  0.09350388, -0.1182863 ,  0.05493767, -0.0540427 ,
       -0.02793282,  0.1446246 ,  0.06644139,  0.21412279, -0.04277317,
        0.04283172, -0.29537287, -0.12718056, -0.17874782, -0.05980247,
        0.1844788 ,  0.20496762,  0.01548754,  0.20056264,  0.10952497,
        0.22693874, -0.03669243, -0.28587013, -0.00604952, -0.19792321,
        0.12397663,  0.02489714, -0.03252879, -0.21052079, -0.02647402,
       -0.02200038,  0.05241749,  0.02099624,  0.00968245,  0.0363222 ,
       -0.19844279, -0.09235109, -0.01037767,  0.00188219, -0.17111456,
        0.13623615, -0.11319786, -0.12628049, -0.13340236,  0.00768357,
        0.20510332,  0.313057  , -0.01723125,  0.01958176, -0.0440247 ,
        0.31768644,  0.04092257,  0.15790057, -0.21108064,  0.16